In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ubfc_datatape_160x36x36_train.h5"
valid_tape = "Z:/rppg/ubfc_datatape_160x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_tscan(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_tscan(train).cache(f'{tmp}/tscan'), to_tf_tscan(valid).cache(f'{tmp}/tscan')
train, valid = to_tf_tscan(train), to_tf_tscan(valid)

In [3]:
ts_can_e2e = TS_CAN_end_to_end(n=20)
ts_can_e2e.compile(optimizer='Nadam', loss='mse')
ts_can_e2e.build(input_shape=(None, 36, 36, 3))

In [4]:
ts_can_e2e.fit(train.shuffle(2000), validation_data=valid, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/TS-CAN_UBFC.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
ts_can_e2e.load_weights('../weights/TS-CAN_UBFC.h5')

Epoch 1/10
485/485 [==============================] - 19s 27ms/step - loss: 0.2529 - val_loss: 0.2168

Epoch 00001: val_loss improved from inf to 0.21684, saving model to ../weights\TS-CAN_UBFC.h5
Epoch 2/10
485/485 [==============================] - 16s 26ms/step - loss: 0.2454 - val_loss: 0.2254

Epoch 00002: val_loss did not improve from 0.21684
Epoch 3/10
485/485 [==============================] - 16s 26ms/step - loss: 0.2395 - val_loss: 0.2231

Epoch 00003: val_loss did not improve from 0.21684
Epoch 4/10
485/485 [==============================] - 16s 27ms/step - loss: 0.2351 - val_loss: 0.2125

Epoch 00004: val_loss improved from 0.21684 to 0.21251, saving model to ../weights\TS-CAN_UBFC.h5
Epoch 5/10
485/485 [==============================] - 16s 27ms/step - loss: 0.2312 - val_loss: 0.2178

Epoch 00005: val_loss did not improve from 0.21251
Epoch 6/10
485/485 [==============================] - 27s 48ms/step - loss: 0.2267 - val_loss: 0.2355

Epoch 00006: val_loss did not improve

In [5]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [6]:
eval_on_dataset(test_set_UBFC_rPPG2, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_UBFC_PURE.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/TS-CAN_UBFC_PURE.h5')
get_metrics('../results/TS-CAN_UBFC_PURE.h5')

100%|██████████| 42/42 [00:37<00:00,  1.11it/s]


{'Sliding window': {'MAE': 4.58, 'RMSE': 10.421, 'R': 0.82538},
 'Whole video': {'MAE': 4.789, 'RMSE': 11.87, 'R': 0.77325}}